# API

## /idx/stock-summary

In [ ]:
from datetime import date, timedelta
from time import sleep

import httpx

from voxrow.web.domain.value_objects import Settings

In [ ]:
start_date: date = date(2026, 1, 20)
settings: Settings = Settings()

for days in range((date.today() - start_date).days + 1):
    current_date: date = start_date + timedelta(days=days)
    resp: httpx.Response = httpx.get(
        f"https://voxrow.com/idx/stock-summary?date={current_date}",
        headers=dict(Authorization=f"Bearer {settings.cron_secret.get_secret_value()}"),
        timeout=60 * 1.5,
    )

    print(f"{current_date}: {resp.status_code}")
    sleep(15)

# Duckdb

## Preparation

In [ ]:
from os import environ

from voxrow.web.domain.value_objects import Settings

In [ ]:
settings: Settings = Settings()
environ["AWS_ACCESS_KEY_ID"] = settings.cloudflare_r2.aws_access_key_id.get_secret_value()
environ["AWS_SECRET_ACCESS_KEY"] = settings.cloudflare_r2.aws_secret_access_key.get_secret_value()
ACCOUNT_ID: str = settings.cloudflare_r2.endpoint_url.host.split(".")[0]

In [ ]:
%load_ext magic_duckdb
%dql -t relation
%dql SELECT 'Hanya Tes' ini;

In [ ]:
%%dql -j
CREATE OR REPLACE
SECRET (
    TYPE r2,
    PROVIDER credential_chain,
    CHAIN env,
    ACCOUNT_ID '{{ ACCOUNT_ID }}'
);

## Queries

In [ ]:
%%dql
SELECT
    *
FROM
    'r2://datalake/idx.co.id/GetStockSummary/*.json'
;